In [ ]:
#DEVANSH TIWARI

In [ ]:
#Ans - 1

agent1.sources =source1
agent1.sinks = sink1
agent1.channels = channel1

agent1.sources.source1.channels = channel1
agent1.sinks.sink1.channel = channel1

agent1.sources.source1.type = spooldir
agent1.sources.source1.spoolDir = /home/devansh/spooldir

agent1.sinks.sink1.type = hdfs
agent1.sinks.sink1.hdfs.path = /flume
agent1.sinks.sink1.hdfs.filePrefix = events
agent1.sinks.sink1.hdfs.fileSuffix = .log
agent1.sinks.sink1.hdfs.inUsePrefix = _
agent1.sinks.sink1.hdfs.fileType = DataStream

agent1.channels.channel1.type = file

In [ ]:
#Ans - 2

sqoop import \
–connect jdbc:mysql://localhost/empdb \
--username userid \
--password pwd \
--table emp \
--direct \
-m 1 \
--hive-import \
--create-hive-table \
--hive-table emp_mysql \
--target-dir /user/hive/warehouse/ \
--enclosed-by '\"' \
--fields-terminated-by , \
--escaped-by \\ \

In [ ]:
#Ans - 3

sc
airlinesPath="hdfs:///spark/rdd/airlines.csv"
airlinesPath
airlines=sc.textFile(airlinesPath)
airlines.take(10)
airlines.collect()
airlines.count()
airlines.first()

In [ ]:
#Ans - 4

sc
twitterPath="hdfs:///spark/cache-0.json"
import json
twitterData=sc.textFile(twitterPath).map(lambda x:json.loads(x))
from pyspark.sql import SQLContext, Row
sqlC=SQLContext(sc)
sqlC
sqlC=SQLContext(sc)
twitterTable=sqlC.read.json(twitterPath)
twitterTable.registerTempTable("twitterTable")
sqlC.sql("Select text, user.screen_name from twitterTable where user.screen_name='realDonaldTrump' limit 10").collect()

In [ ]:
#Ans - 5

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext("local[2]","StreamingErrorCount")
ssc = StreamingContext(sc,10)
ssc.checkpoint("/streaming")
ds1 = ssc.socketTextStream("localhost",9999)
count = ds1.flatMap(lambda x:x.split(" ")).filter(lambda word:"Error" in word).map(lambda word:(word,1)).reduceByKey(lambda x,y:x+y)

count.pprint()
ssc.start()
ssc.awaitTermination()

In [ ]:
#Ans - 6

sc
flightsPath = "hdfs:///spark/rdd/flights.csv"
flightsData = sc.textFile(flightsPath)
f = finalData.filter(notHeader).map(split)
fpar.filter(lambda x:x.dep_delay>0).count()/float(fpar.count())
sumCount=fpar.map(lambda x:x.dep_delay).aggregate((0,0),
                                                          (lambda acc,value: (acc[0]+value, acc[1]+1)),
                                                          (lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])))
print "Average delay is "+str(sumCount[0]/float(sumCount[1]))

In [ ]:
#Ans - 7

sc
flightsPath = "hdfs://localhost:9000/flights.csv"
flightsData = sc.textFile(flightsPath)

def notHeader(row):
    return "AIRLINE_ID" not in row

import csv 
from StringIO import StringIO
def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

d1 = flightsData.filter(notHeader)
d2 = d1.map(split)

from datetime import datetime 
d3 = d2.map(lambda x: Row(date= datetime.strptime(x[0], "%Y-%m-%d"), airline = x[1], flightnum=x[2], origin=x[3], dest=x[4], dep=x[5],
                                                              dep_delay=x[6], arv=x[7], arv_delay=x[8],
                                                               airtime=x[9], distance=float(x[10]) ))
d3.\
    filter(lambda x: (x['dest']== "LAX")  and (x['date'] == datetime.strptime("2016-01-01", "%Y-%m-%d"))).\
    count()

In [ ]:
#Ans - 8

sc
flightsPath = "hdfs:///spark/rdd/flights.csv"
flightsData = sc.textFile(flightsPath)

from datetime import datetime 
from collections import namedtuple

fields   = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep',
            'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight   = namedtuple('Flight', fields, verbose=False)

DATE_FMT = '%Y-%m-%d'

TIME_FMT = '%H%M%S' # %S is added to remove error "unconverted data remains 0 " in strptime()


def parse(row):
    row[0]  = datetime.strptime(row[0], DATE_FMT).date()
    row[5]  = datetime.strptime(row[5], TIME_FMT).time()
    row[6]  = float(row[6])
    row[7]  = datetime.strptime(row[7], TIME_FMT).time()
    row[8]  = float(row[8])
    row[9]  = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

flightsData.first()
blanks = flightsData.map(lambda x:','.join(x or '00.00' for x in x.split(',')))
blanktime = blanks.map(lambda x:x.replace(',""' , ',"0000"') )
finalFlights = blanktime